In [1]:
import os
from datasets import load_dataset
from PIL import Image
import pandas as pd
from easyocr import Reader
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
import pytesseract


In [2]:
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)     # Show all rows
pd.set_option('display.max_colwidth', None) # Show full content of each column (no truncation)
pd.set_option('display.width', None)

In [3]:
dataset = load_dataset("ahmedheakl/arocrbench_khatt",split="train")


In [4]:
results = []
for sample in dataset:
    image = sample["image"]  # Already a PIL Image object
    if image.mode != "RGB":
        image = image.convert("RGB")
    ground_truth = sample["text"]
    results.append({
        "image": image,
        "ground_truth": ground_truth,
        "tesseract": None,
        "easyocr": None,
    })
print(f"Initialized results with {len(results)} samples.")

Initialized results with 200 samples.


### Tesseract Processing

In [5]:
for i, sample in enumerate(results):
    image = sample["image"]
    tesseract_text = pytesseract.image_to_string(image,lang='ara')
    results[i]['tesseract'] = tesseract_text
print("Tesseract Processing completeed")

Tesseract Processing completeed


## EasyOCR

In [6]:
easyocr_reader = Reader(['ar'], gpu=True)
for i, sample in enumerate(results):
    image = sample["image"]
    easyocr_result = easyocr_reader.readtext(image, detail=0)
    easyocr_text = " ".join(easyocr_result)
    results[i]["easyocr"] = easyocr_text
print("EasyOCR processing completed.")

EasyOCR processing completed.


In [7]:
df = pd.DataFrame(results)
df.head()


,image,ground_truth,tesseract,easyocr
0,<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=2142x141 at 0x7D6DD319BF90>,وصفاً مجملاً لحالة المغرب خلال سنة 2005، مثيراً ، في سياق ذلك ، التساؤلات الكبرى,"و اث\nحلم خالد\nصف للع نب 1\n‎١‏ حال ل 0\n2ه ل\n2005 ضرا ء :\n- ضيبا ماد لأى مع\nالمشساز ,لي\n8 1""\n","حالا مح٥ً لحالدالمح صف سراء ذ سب مه دلل ا ,لسادلان للورا ها )4~5ه2"
1,<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=2159x228 at 0x7D6DD1F86490>,وهلال خازن عفيف للحج . بدأت قوافل الحجيج حاج إثر آخر يلبي,"“ان طازن, عض للهبع “د أات اطع لجيج ما رك نم مل\n",عصعا معبل ل _ جا ت قواقل عا 2 إثر ~ خ ما رره للهج المجيج
2,<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1508x404 at 0x7D6DD1F87E50>,النص: مشمش ، دراق ، غيظ ، ناء ، بث ، نسر .,الى : محش د راك حيظ ؛ نأد؛ جمبثت اجر\n,"لبث دراد , خيظ ' نا ء مهمم الذم نسلن"
3,<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1236x103 at 0x7D6DD1FA03D0>,و هي التي جعلته كريما على فقره .,هو هى انوا كلت دريما كل موه .\n,لريما ىلى ححلد الد مشظو ه
4,<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1873x110 at 0x7D6DD1FA0950>,مثل أنقض بغـلس له الضابط لزمتك. سألت راجح هل أبلغ أصحابنا,ملا دمض نغلس هف الطابط لزمتك ساهه راجع هل أبلؤ أصمجاين!١‏\n,امجا سالت راجح هل لل الطا ط مث ' نقحى ابل بنا لرمرك نغلل


In [8]:
from jiwer import wer, cer 

for model in ["tesseract","easyocr"]:
    wer_score = wer(df["ground_truth"].tolist(), df[model].tolist())
    cer_score = cer(df["ground_truth"].tolist(), df[model].tolist())
    print(f"{model} - WER: {wer_score:.2f}, CER: {cer_score:.2f}")

tesseract - WER: 1.17, CER: 0.66
easyocr - WER: 1.06, CER: 0.69


In [9]:
df.to_csv("khattat_result.csv")
